In [13]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

### enter the car link directly from auction

In [30]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(r"D:\GS_WebScarping\project_10_newark\chromedriver.exe")
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    
    

    results = []
    
   # car name and address --> done
    while True:
        details = {}
        try:
            top_main_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="panel-heading watchlist-container"]')))
            # driver.refresh()
            if top_main_card:
                
                # get the car name
                try:
                    title = WebDriverWait(top_main_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="h4 panel-title "]')))
                    if title:
                        details['Name'] = title.text.strip()
                    else:
                        details['Name']='na'
                except Exception as e:
                    print("No car name")
                    
                # location
                try:
                    car_loc= WebDriverWait(top_main_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="h6 no-margin-bottom"]')))
                    if car_loc:
                        details['Location'] = car_loc.text.strip()
                    else:
                        details['Location'] = "na"
                except Exception as e:
                    print("No car location")
                    
            else:
                print("no main top card")
        except Exception as e:
            print("no main top card")
            
        # images --> done
        try:
            main_img_card =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="col-lg-5 col-md-5 col-sm-12 col-xs-12"]'))) 
            if main_img_card:
                imgs = WebDriverWait(main_img_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class="img-responsive btn-block"]')))
                # imgs = imgs_card.find_elements(By.TAG_NAME, "img")
                # imgs_list = []
                if imgs:
                    imgs_list = [img.get_attribute("data-lazy") or img.get_attribute("src") for img in imgs]
                imgs_str = ", ".join(imgs_list)
                details['Images'] = imgs_str
            else:
                print("No images card")
        except Exception as e:
            print("No images card")
        
        # get odometer --> done
        try:
            odo = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/p[@class="no-margin"]'))).text.strip()
            if odo:
                odo_met = odo.split(":")
                details['Odometer'] = odo_met[1]
            else:
                details['Odometer']  = 'na'
        except Exception as e:
            print("No odometer")
            
            
            
        # specifictaion data --> done
        try:
            specs_main = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="col-lg-7 col-md-7 col-sm-12 col-xs-12 no-padding-left-lg specifications"]')))
            if specs_main:
                specs =WebDriverWait(specs_main, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="clearfix text-right"]')))
                if specs:
                    for spec in specs:
                        spec_lbl = spec.find_element(By.XPATH, './/p[@class="pull-left"]').text.strip()
                        spec_val = spec.find_element(By.CSS_SELECTOR,'.inline').text.strip()
                        if spec_lbl and spec_val:
                            details[spec_lbl] = spec_val
                        else:
                            print("No specs details")
            else:
                print("No specs card")
        except Exception as e:
            print("No specs card")
        
        
        
        # desc --> done
        try:
            descs = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/legend[@class="text-primary h4"]')))
            if len(descs) > 1:
                dessc_lst = [desc.text.strip() for desc in descs]
                details['Description'] = dessc_lst[1]
            else:
                details['Description'] = "na"
        except Exception as e:
            print("No description")
            
            
        # features and comfort --> done
        try:
            feat_com_main =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="row grid is-active"]'))) 
            if feat_com_main:
                feat_vals = feat_com_main.find_elements(By.XPATH, './/div[@class="text-justify"]')
                if feat_vals:
                    feat_vals_lst = [feat.text.strip() for feat in feat_vals]
                    details['Features'] = feat_vals_lst
                else:
                    details['Features'] = 'na'
        except Exception as e:
            print("No feature found")

        
        # report --> not done bcz no direct link to the pdf given
        # try:
        #     report_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="panel-heading header-content"]')))
        #     if report_card:
        #         repo_button = report_card.find_element(By.XPATH, './/div[@class="btn btn-default btn-print popover-btn"]')
        #         if repo_button:
        #             repo_button.click()
        #             time.sleep(3)
                    
        #             repo_pdf = driver.find_element(By.XPATH, './/div[@id="pdf"]')
        #             if repo_pdf:
        #                 pdf_linik = repo_pdf.get_attribute("href")
        #                 details['report'] = pdf_linik
        #             else:
        #                 details['report'] = 'na'
        # except Exception as e:
        #     print("No report")
        
        results.append(details)
        
        # get the next button 
        try:
            next_button_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="input-group selectpicker-wrap"]')))
            if next_button_card:
                next_btn = WebDriverWait(next_button_card, 10).until(EC.presence_of_element_located((By.XPATH, './/a[text()="»"]')))
                if next_btn:
                    # driver.execute_script("arguments[0].scrollIntoView();", next_btn)
                    time.sleep(2)
                    next_btn.click()
                    driver.refresh()
                else:
                    print("No next button hence no more cars are present")
        except Exception as e:
            print("No next button hence no more cars are present")
            
            break
        
    # results.append(details)
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("newark.csv", index=False)       
    time.sleep(5)     
    driver.quit()
path = "https://www.newarkmotorauctions.co.uk/ford-s-max-zetec-tdci-zetec-tdci-other-newark-coddington_vid_13875"
scrape(path)

No specs details
No feature found
No specs details
No feature found
No specs details
No feature found
No specs details
No feature found
No specs details
No feature found
No specs details
No feature found
No specs details
No feature found
No specs details
No specs details
No specs details
No next button hence no more cars are present


In [31]:
df= pd.read_csv("newark.csv")
df

,Name,Location,Images,Odometer,Reg No.,MOT,V5,Vat Status,Body Style,Engine,Gearbox,Stock #,1st Reg,Chassis No,Fuel Type,Drive Type,Exterior,Interior,Description,Features
0,2014 FORD S-MAX ZETEC TDCI ZETEC TDCI,"Lincoln Rd, NEWARK, CODDINGTON",https://www.newarkmotorauctions.co.uk/image/13...,"79,681 mi",YW14CPK,2024-01-31,Here,STANDARD - NO VAT,OTHER,4 CYLINDERS,6 SPEED MANUAL DIESEL,500502,2014-08-01,WF0SXXGBWSEM75867,Diesel,4X2,BLUE,OTHER,na,NaN
1,2014 FORD FIESTA TITANIUM ECONETIC TITANIUM EC...,"Lincoln Rd, NEWARK, CODDINGTON",https://www.newarkmotorauctions.co.uk/image/13...,"72,841 mi",AK64NAU,2025-03-20,No,STANDARD - NO VAT,HATCHBACK,4 CYLINDERS,5 SPEED MANUAL DIESEL,500447,2014-12-01,WF0DXXGAKDEG60535,Diesel,4X2,WHITE,OTHER,na,NaN
2,2015 VAUXHALL ASTRA ELITE ELITE,"Lincoln Rd, NEWARK, CODDINGTON",https://www.newarkmotorauctions.co.uk/image/13...,"110,782 mi",LD15PGO,2024-12-09,No,STANDARD - NO VAT,OTHER,4 CYLINDERS,5 SPEED MANUAL PETROL,500445,2015-05-01,W0LPE8ED6F8054163,PETROL,4X2,BROWN,OTHER,na,NaN
3,2015 KIA RIO 2 ISG CRDI CRDI 2 ISG,"Lincoln Rd, NEWARK, CODDINGTON",https://www.newarkmotorauctions.co.uk/image/14...,"101,462 mi",LB15XVU,2024-09-19,No,STANDARD - NO VAT,HATCHBACK,3 CYLINDERS,6 SPEED MANUAL DIESEL,500565,2015-07-01,KNADN515LF6717875,Diesel,4X2,BLACK,OTHER,na,NaN
4,2019 VAUXHALL GRANDLAND X ELITE NAV T D ELITE ...,"Lincoln Rd, NEWARK, CODDINGTON",https://www.newarkmotorauctions.co.uk/image/14...,"999,999 mi",EN19RKO,2025-05-19,No,STANDARD - NO VAT,HATCHBACK,4 CYLINDERS,6 SPEED MANUAL DIESEL,500531,2019-05-01,W0VZCYHZ5KS172814,Diesel,4X2,GREY,OTHER,na,NaN
5,2013 VAUXHALL CORSA SE SE,"Lincoln Rd, NEWARK, CODDINGTON",https://www.newarkmotorauctions.co.uk/image/14...,"94,272 mi",DU13FVT,2025-05-10,Here,STANDARD - NO VAT,HATCHBACK,4 CYLINDERS,5 SPEED MANUAL PETROL,500574,2013-03-01,W0L0SDL68D4107318,PETROL,4X2,SILVER,OTHER,na,NaN
6,2017 CITROEN C3 FLAIR PURETECH PURETECH FLAIR,"Lincoln Rd, NEWARK, CODDINGTON",https://www.newarkmotorauctions.co.uk/image/14...,1 mi,AJ17OLK,2025-05-06,No,STANDARD - NO VAT,HATCHBACK,3 CYLINDERS,5 SPEED MANUAL PETROL,500560,2017-06-01,VF7SXHMZ6HT609614,PETROL,4X2,BLUE,OTHER,na,NaN
7,2014 NISSAN NV200 ACENTA DCI DCI ACENTA,"Lincoln Rd, NEWARK, CODDINGTON",https://www.newarkmotorauctions.co.uk/image/14...,"125,116 mi",DL64PVV,2024-07-30,No,STANDARD - NO VAT,VAN,4 CYLINDERS,5 SPEED MANUAL DIESEL,500530,2014-12-01,VSKYBAM20U0092240,Diesel,4X2,WHITE,OTHER,EQUIPMENT AND OPTIONS OF THIS NISSAN NV200 ACE...,"['Drivable, RevC, SatN']"
8,2015 VAUXHALL CORSA DESIGN CDTI ECOFLEX DESIGN...,"Lincoln Rd, NEWARK, CODDINGTON",https://www.newarkmotorauctions.co.uk/image/13...,"79,542 mi",WR65YJZ,2025-09-27,Here,STANDARD - NO VAT,HATCHBACK,ENGINE,5 SPEED MANUAL DIESEL,500352,2015-11-01,W0L0XEP68F4293163,Diesel,4X2,BLACK,OTHER,EQUIPMENT AND OPTIONS OF THIS VAUXHALL CORSA D...,"['Cruise Control', 'DABR, Drivable']"
9,2013 VOLVO V40 ES D2 D2 ES,"Lincoln Rd, NEWARK, CODDINGTON",https://www.newarkmotorauctions.co.uk/image/13...,"160,706 mi",YB13LFZ,2025-01-25,Here,STANDARD - NO VAT,HATCHBACK,ENGINE,6 SPEED MANUAL DIESEL,500344,2013-07-01,YV1MV8481E2088617,Diesel,4X2,RED,OTHER,EQUIPMENT AND OPTIONS OF THIS VOLVO V40 ES D2,"['Cruise Control', 'DABR, Drivable']"


In [34]:
df['Images'][7]

'https://www.newarkmotorauctions.co.uk/image/14009_w640_h480_0.jpg, /image/14009_w640_h480_0.jpg, /image/14009_w100_h75_0.jpg'

In [35]:
reg_img=df[['Reg No.', 'Images']]

In [ ]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg No."] # separate reg nums
        image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)
